## Testing The AWS HTTP API Endpoint

After deploying the endpoint application to AWS as specified in this distribution's README file, this notebook can be used to test it.

### Specify the API Gateway URL

In the AWS console, go to the API Gateway service, select the deployed API, and go to the API details screen.  Copy the invocation URL and set the variable below to the URL. The URL in API Gateway will have the format specified below.

In [ ]:
api = 'https://<An AWS randomly generated ID>.execute-api.<ACCOUNT REGION>.amazonaws.com'

### Call GET on Lambda Function

Lets test the GET route. The response is constructed in the `app.py` file's `lambda_handler` function.

In [2]:
import requests

url = api + '/getInfo'
response = requests.get(url)
print(f'{response.status_code=}')
print(f'{response.text=}')


response.status_code=200
response.text='"Get info called!"'


### Post Image to Lambda Function - Post An Image File

Now test the `/upload` route. The Lambda handler code expects the body of the post to be an image. The handler extracts, decodes, and posts the image to the S3 bucket specified in the `.env` found int the `upload` folder. The image in the S3 bucket will have the key `IMG_{YYYMMDD_HHMMSS}.png`.

In [3]:
import requests

url = api + '/upload'
file_path = './sunflower1.jpg'

with open(file_path, 'rb') as image_file:
    response = requests.post(url, data=image_file)

if response.status_code == 200:
    print("Image uploaded successfully!")
else:
    print("Error uploading image:", response.status_code)

print(f'{response.status_code=}')
print(f'{response.text=}')

Image uploaded successfully!
response.status_code=200
response.text='Detected labels:\nLabel: Sunflower, Conf: 100.0\nLabel: Daisy, Conf: 71.3'


### Post Image to Lambda Function - Post JSON document containing Image 

Now lets test the `/upload2` route. This is similar to the `/upload` route, except the handler code for this route expects the body to be a JSON encoded Python dictionary. Once decoded, the image is retrieved from the dictionary and posted to the S3 bucket.

In [8]:
import requests
import base64
import json
import base64
import json

def serialize_image(file_path):
    with open(file_path, 'rb') as image_file:
        encoded_image = base64.b64encode(image_file.read()).decode('utf-8') 
    data = {'image': encoded_image}
    json_string = json.dumps(data)
    return json_string

url = api + '/upload2'
file_path = "./bkr.png"

data = serialize_image(file_path)
response = requests.post(url, data=data)

if response.status_code == 200:
    print("Image uploaded successfully!")
else:
    print("Error uploading image:", response.status_code)

print(f'{response.status_code=}')
print(f'{response.text=}')

Image uploaded successfully!
response.status_code=200
response.text='"Post 2 called!"'
